<a href="https://colab.research.google.com/github/Harisonm/4aibd-s1-project-ml/blob/yas_branch/4aibd-s1-project-ml/default/apps/notebooks/Rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# CNN RNN model for CIFAR-10
import numpy
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Reshape
from keras.layers import TimeDistributed, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.constraints import maxnorm
from keras.optimizers import SGD
from keras.utils import np_utils
from keras import backend as K
from keras.callbacks import TensorBoard

Using TensorFlow backend.


In [0]:
K.set_image_dim_ordering('tf')

In [0]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

batch_size = 128

In [6]:
# load data
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 39s 0us/step


In [0]:
# normalize inputs from 0-255 to 0.0-1.0
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train = X_train / 255.0
X_test = X_test / 255.0

In [0]:
# one hot encode outputs
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [9]:
# Create the model
model = Sequential()
# define CNN model
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), 
                    activation='relu', padding='same'))
#model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
#model.add(TimeDistributed(Flatten()))
model.add(Reshape((32, 32*32)))
# define LSTM model
model.add(LSTM(32, 
                    activation='relu', kernel_constraint=maxnorm(3),
                    return_sequences=True
              ))
model.add(Dropout(0.2))
model.add(LSTM(32, activation='relu', kernel_constraint=maxnorm(3)))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
# Compile model
epochs = 25
lrate = 0.01
decay = lrate/epochs
sgd = SGD(lr=lrate, momentum=0.9, decay=decay, nesterov=False)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
print(model.summary())

In [0]:
model_str = "RNN_" + str(epochs) + "_epochs_" + "cifar10" + "categorical_crossentropy"

#model.save("./tensorboard/saved_models" + model_str, True, True)

In [0]:
# Tensorboard callback
tb_callback = TensorBoard(
    log_dir="./tensorboard/logs/" + "RNN" + "_epochs_" + str(epochs) + "RNN_same_relu_softmax")

In [0]:
# Fit the model
# batch_size -> a modifier selon la carte graphique

model.fit(X_train, y_train,
          validation_data=(X_test, y_test),
          epochs=epochs,
          batch_size=32)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/25
50000/50000 [==============================] - 242s 5ms/step - loss: 2.2462 - acc: 0.1417 - val_loss: 2.1286 - val_acc: 0.2147
Epoch 2/25
49984/50000 [============================>.] - ETA: 0s - loss: 2.0459 - acc: 0.2235